In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import folium
import datetime
import numpy as np
import pandas as pd
import re
from pytz import timezone
import pytz

In [2]:
def decode_lat_lon(sr: pd.Series) -> pd.Series:
    """
    緯度経度を度数に変換
    """
    tmp = sr.astype(float)
    return (tmp / 100).astype(int) + (tmp % 100) / 60

In [3]:
# LOG = "matsumoto-nagano-100318.log"
LOG = "nagano-matsumoto-100318.log"

gps_table = []

gpgga_list = [0] * 8
gprmc_list = [0] * 9

with open(LOG) as f:
    for line in f:
        str_list = line.split(",")
        if str_list[0] == "$GPGGA":
            gpgga_list = str_list[1:-1]
        elif str_list[0] == "$GPRMC":
            gprmc_list = str_list[1:-3]
            if gpgga_list[0] != gprmc_list[0]:
                print(gpgga_list, gprmc_list)
                gpgga_list = [0] * 8
            elif gpgga_list[1:5] != gprmc_list[2:6]:
                print(gpgga_list, gprmc_list)
                gpgga_list = [0] * 8
            # print(gpgga_list)
            # print(gprmc_list)
            gps_table.append(gprmc_list + gpgga_list[5:])
gps_df = pd.DataFrame(gps_table, columns=[
    "time", "status", "lat", "NS", "lon", "EW",
    "knots", "direction", "date", "quality", "sat",
    "harr", "elevation", "em", "geoid", "gm", "drp_id"
])
gps_df

[0, 0, 0, 0, 0, 0, 0, 0] ['012258', 'A', '3637.9008', 'N', '13811.3569', 'E', '000.3', '004.3', '180310']


,time,status,lat,NS,lon,EW,knots,direction,date,quality,sat,harr,elevation,em,geoid,gm,drp_id
0,012258,A,3637.9008,N,13811.3569,E,000.3,004.3,180310,0,0,0,None,None,None,None,None
1,012312,A,3637.8827,N,13811.4093,E,000.1,060.2,180310,1,05,02.5,00351.5,M,037.4,M,00
2,012327,A,3637.8844,N,13811.4088,E,000.1,173.3,180310,1,05,02.5,00354.3,M,037.4,M,00
3,012342,A,3637.8847,N,13811.4086,E,000.1,041.5,180310,1,05,02.5,00360.4,M,037.4,M,00
4,012357,A,3637.8881,N,13811.4069,E,000.3,141.1,180310,1,04,06.3,00356.6,M,037.4,M,00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,030709,A,3614.3477,N,13758.3616,E,015.8,101.6,180310,1,08,01.0,00597.8,M,037.4,M,00
406,030724,A,3614.3345,N,13758.4639,E,021.7,097.3,180310,1,07,01.2,00599.5,M,037.4,M,00
407,030739,A,3614.3176,N,13758.5716,E,019.7,104.9,180310,1,08,01.0,00595.7,M,037.4,M,00
408,030754,A,3614.3228,N,13758.6188,E,003.9,109.1,180310,1,07,01.2,00604.5,M,037.4,M,00


In [4]:
gps_df["lat_deg"] = decode_lat_lon(gps_df["lat"])
gps_df["lon_deg"] = decode_lat_lon(gps_df["lon"])
gps_df["lat"] = gps_df["lat"] * ((gps_df.pop("NS") == "N") * 2 - 1)
gps_df["lon"] = gps_df["lon"] * ((gps_df.pop("EW") == "E") * 2 - 1)

gps_df["lat"] = np.deg2rad(gps_df["lat_deg"])
gps_df["lon"] = np.deg2rad(gps_df["lon_deg"])

gps_df["sat"] = gps_df["sat"].astype(int)
gps_df["knots"] = gps_df["knots"].astype(float)
gps_df["direction"] = gps_df["direction"].astype(float)
gps_df["harr"] = gps_df["harr"].astype(float)
gps_df["elevation"] = gps_df["elevation"].astype(float)
gps_df["geoid"] = gps_df["geoid"].astype(float)
gps_df["date"] = (gps_df["date"] + gps_df["time"]).apply(lambda x: pytz.timezone("UTC").localize(
    datetime.datetime.strptime(x, "%d%m%y%H%M%S")).astimezone(timezone("Asia/Tokyo")))

gps_df = gps_df.drop(["em", "gm", "time"], axis=1)

gps_df

,status,lat,lon,knots,direction,date,quality,sat,harr,elevation,geoid,drp_id,lat_deg,lon_deg
0,A,0.639343,2.411858,0.3,4.3,2010-03-18 10:22:58+09:00,0,0,0.0,NaN,NaN,None,36.631680,138.189282
1,A,0.639338,2.411873,0.1,60.2,2010-03-18 10:23:12+09:00,1,5,2.5,351.5,37.4,00,36.631378,138.190155
2,A,0.639339,2.411873,0.1,173.3,2010-03-18 10:23:27+09:00,1,5,2.5,354.3,37.4,00,36.631407,138.190147
3,A,0.639339,2.411873,0.1,41.5,2010-03-18 10:23:42+09:00,1,5,2.5,360.4,37.4,00,36.631412,138.190143
4,A,0.639340,2.411873,0.3,141.1,2010-03-18 10:23:57+09:00,1,4,6.3,356.6,37.4,00,36.631468,138.190115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,A,0.632492,2.408078,15.8,101.6,2010-03-18 12:07:09+09:00,1,8,1.0,597.8,37.4,00,36.239128,137.972693
406,A,0.632488,2.408108,21.7,97.3,2010-03-18 12:07:24+09:00,1,7,1.2,599.5,37.4,00,36.238908,137.974398
407,A,0.632483,2.408139,19.7,104.9,2010-03-18 12:07:39+09:00,1,8,1.0,595.7,37.4,00,36.238627,137.976193
408,A,0.632485,2.408153,3.9,109.1,2010-03-18 12:07:54+09:00,1,7,1.2,604.5,37.4,00,36.238713,137.976980


In [5]:
fig = folium.Figure()

center_lat = 36.45
center_lon = 138.0

m = folium.Map([center_lat, center_lon], zoom_start=10).add_to(fig)
for i in gps_df.index:
    folium.Marker(location=[gps_df["lat_deg"][i], gps_df["lon_deg"][i]]).add_to(m)
m

In [6]:
use_columns = ["lat", "lon", "knots", "date", "elevation", "geoid"]

lat_lon_df = gps_df[use_columns][1:].copy() # 先頭削除
lat_lon_df_prev = gps_df[use_columns][:-1].copy() # 末尾削除

for uc in use_columns:
    lat_lon_df[uc + "_prev"] = list(lat_lon_df_prev[uc])
    
lat_lon_df

,lat,lon,knots,date,elevation,geoid,lat_prev,lon_prev,knots_prev,date_prev,elevation_prev,geoid_prev
1,0.639338,2.411873,0.1,2010-03-18 10:23:12+09:00,351.5,37.4,0.639343,2.411858,0.3,2010-03-18 10:22:58+09:00,NaN,NaN
2,0.639339,2.411873,0.1,2010-03-18 10:23:27+09:00,354.3,37.4,0.639338,2.411873,0.1,2010-03-18 10:23:12+09:00,351.5,37.4
3,0.639339,2.411873,0.1,2010-03-18 10:23:42+09:00,360.4,37.4,0.639339,2.411873,0.1,2010-03-18 10:23:27+09:00,354.3,37.4
4,0.639340,2.411873,0.3,2010-03-18 10:23:57+09:00,356.6,37.4,0.639339,2.411873,0.1,2010-03-18 10:23:42+09:00,360.4,37.4
5,0.639340,2.411872,0.2,2010-03-18 10:24:12+09:00,358.6,37.4,0.639340,2.411873,0.3,2010-03-18 10:23:57+09:00,356.6,37.4
...,...,...,...,...,...,...,...,...,...,...,...,...
405,0.632492,2.408078,15.8,2010-03-18 12:07:09+09:00,597.8,37.4,0.632495,2.408060,4.8,2010-03-18 12:06:54+09:00,599.0,37.4
406,0.632488,2.408108,21.7,2010-03-18 12:07:24+09:00,599.5,37.4,0.632492,2.408078,15.8,2010-03-18 12:07:09+09:00,597.8,37.4
407,0.632483,2.408139,19.7,2010-03-18 12:07:39+09:00,595.7,37.4,0.632488,2.408108,21.7,2010-03-18 12:07:24+09:00,599.5,37.4
408,0.632485,2.408153,3.9,2010-03-18 12:07:54+09:00,604.5,37.4,0.632483,2.408139,19.7,2010-03-18 12:07:39+09:00,595.7,37.4


### [国土地理院の計算方法](https://vldb.gsi.go.jp/sokuchi/surveycalc/surveycalc/algorithm/bl2st/bl2st.htm)

In [7]:
# GRS80楕円体
a = 6378.137 # 長半径 (km)
f = 1 / 298.257222101 # 扁平率

exception_zones = ("2", "3a", "3b1")

def calc_dist_kokudo(phi1, L1, phi2, L2):
    l = L2 - L1
    if l > np.pi:
        l_prime = l - 2 * np.pi
    elif l < -np.pi:
        l_prime = l + 2 * np.pi
    else:
        l_prime = l
    L = abs(l_prime)
    L_prime = np.pi - L
    if l_prime >= 0:
        u1 = np.arctan((1 - f) * np.tan(phi1))
        u2 = np.arctan((1 - f) * np.tan(phi2))
        delta = phi2 - phi1
    else:
        u1 = np.arctan((1 - f) * np.tan(phi2))
        u2 = np.arctan((1 - f) * np.tan(phi1))
        delta = phi1 - phi2
    SIGMA = phi1 + phi2
    SIGMA_prime = u1 + u2
    delta_prime = u2 - u1
    xi = np.cos(SIGMA_prime / 2)
    xi_prime = np.sin(SIGMA_prime / 2)
    eta = np.sin(delta_prime / 2)
    eta_prime = np.cos(delta_prime / 2)
    
    x = np.sin(u1) * np.sin(u2)
    y = np.cos(u1) * np.cos(u2)
    c = y * np.cos(L) + x
    epsilon = f * (2 - f) / (1 - f) ** 2
    
    if c >= 0:
        theta = L * (1 + f * y)
        zone = "1"
        
    if zone != "1":
        print("未実装")
        return -1
    F = 1
    while abs(F) >= 1e-15:
        if zone in exception_zones:
            g = np.sqrt((eta * np.sin(theta / 2)) ** 2 + (xi * np.cos(theta / 2)) ** 2)
            h = np.sqrt((eta_prime * np.sin(theta / 2)) ** 2 + (xi_prime * np.cos(theta / 2)) ** 2)
        else:
            g = np.sqrt((eta * np.cos(theta / 2)) ** 2 + (xi * np.sin(theta / 2)) ** 2)
            h = np.sqrt((eta_prime * np.cos(theta / 2)) ** 2 + (xi_prime * np.sin(theta / 2)) ** 2)
        
        sigma = 2 * np.arctan(g / h)
        J = 2 * g * h
        K = h ** 2 - g ** 2
        gamma = y * np.sin(theta) / J
        GAMMA = 1 - gamma ** 2
        zeta = GAMMA * K - 2 * x
        zeta_prime = zeta + x
        D = f * (1 + f) / 4 - 3 * f ** 2 * GAMMA / 16
        E = (1 - D * GAMMA) * f * gamma * (sigma + D * J * (zeta + D * K * (2 * zeta ** 2 - GAMMA ** 2)))
        if zone in exception_zones:
            F = theta - L_prime + E
        else:
            F = theta - L - E
        G = f * gamma ** 2 * (1 - 2 * D * GAMMA) + f * zeta_prime * (sigma / J) * (1 - D * GAMMA + f * gamma ** 2 / 2)\
            + f ** 2 * zeta * zeta_prime / 4
        theta = theta - F / (1 - G)
#         print(F, theta)
    
    n0 = epsilon * GAMMA / (np.sqrt(1 + epsilon * GAMMA) + 1) ** 2
    A = (1 + n0) * (1 + 5 * n0 ** 2 / 4)
    B = epsilon * (1 - 3 * n0 ** 2 / 8) / (np.sqrt(1 + epsilon * GAMMA) + 1) ** 2
    s = (1 - f) * a * A * (sigma - B * J * (zeta - B * (K * (GAMMA ** 2 - 2 * zeta ** 2)\
        - B * zeta * (1 - 4 * K ** 2) * (3 * GAMMA ** 2 - 4 * zeta ** 2) / 6) / 4))
    return s

In [8]:
dist_list = []
for idx, sr in lat_lon_df.iterrows():
    dist_list.append(calc_dist_kokudo(sr["lat"], sr["lon"], sr["lat_prev"], sr["lon_prev"]))
lat_lon_df["dist_kokudo"] = dist_list
lat_lon_df

,lat,lon,knots,date,elevation,geoid,lat_prev,lon_prev,knots_prev,date_prev,elevation_prev,geoid_prev,dist_kokudo
1,0.639338,2.411873,0.1,2010-03-18 10:23:12+09:00,351.5,37.4,0.639343,2.411858,0.3,2010-03-18 10:22:58+09:00,NaN,NaN,0.084982
2,0.639339,2.411873,0.1,2010-03-18 10:23:27+09:00,354.3,37.4,0.639338,2.411873,0.1,2010-03-18 10:23:12+09:00,351.5,37.4,0.003231
3,0.639339,2.411873,0.1,2010-03-18 10:23:42+09:00,360.4,37.4,0.639339,2.411873,0.1,2010-03-18 10:23:27+09:00,354.3,37.4,0.000630
4,0.639340,2.411873,0.3,2010-03-18 10:23:57+09:00,356.6,37.4,0.639339,2.411873,0.1,2010-03-18 10:23:42+09:00,360.4,37.4,0.006780
5,0.639340,2.411872,0.2,2010-03-18 10:24:12+09:00,358.6,37.4,0.639340,2.411873,0.3,2010-03-18 10:23:57+09:00,356.6,37.4,0.003326
...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,0.632492,2.408078,15.8,2010-03-18 12:07:09+09:00,597.8,37.4,0.632495,2.408060,4.8,2010-03-18 12:06:54+09:00,599.0,37.4,0.094527
406,0.632488,2.408108,21.7,2010-03-18 12:07:24+09:00,599.5,37.4,0.632492,2.408078,15.8,2010-03-18 12:07:09+09:00,597.8,37.4,0.155196
407,0.632483,2.408139,19.7,2010-03-18 12:07:39+09:00,595.7,37.4,0.632488,2.408108,21.7,2010-03-18 12:07:24+09:00,599.5,37.4,0.164354
408,0.632485,2.408153,3.9,2010-03-18 12:07:54+09:00,604.5,37.4,0.632483,2.408139,19.7,2010-03-18 12:07:39+09:00,595.7,37.4,0.071365


### 球の接平面近似

In [10]:
# 地球の長半径
R = 6378.137

# 最大値 (距離は最小値)
example_lat = lat_lon_df["lat"].max()

dist_per_lat = R 

0.6393703627850789